# Libraries

In [1]:
library(TFBSTools)
library(Biostrings)
library(JASPAR2020)
library(seqinr)



Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Lade nötiges Paket: S4Vectors

Lade nötiges Paket: stats4


Attache Paket: ‘S4Vectors’


Die folgenden Objekte sind maskiert von ‘package:base’:

    I, expand.grid, unname


Lade nötiges Paket: IRanges

Lade nötiges Paket: XVector

Lade nötiges Paket: GenomeInfoDb


Attache Paket: ‘Biostrings’


Das folgende Objekt ist maskiert ‘package:base’:

    strsplit



Attache Paket: ‘seqi

In [2]:
sessionInfo()

R version 4.2.2 (2022-10-31)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Catalina 10.15.7

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.2/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.2/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] seqinr_4.2-16       JASPAR2020_0.99.10  Biostrings_2.66.0  
[4] GenomeInfoDb_1.34.9 XVector_0.38.0      IRanges_2.32.0     
[7] S4Vectors_0.36.2    BiocGenerics_0.44.0 TFBSTools_1.36.0   

loaded via a namespace (and not attached):
 [1] bitops_1.0-7                matrixStats_1.0.0          
 [3] DirichletMultinomial_1.40.0 bit64_4.0.5                
 [5] httr_1.4.7                  repr_1.1.4                 
 [7] tools_4.2.2                 utf8_1.2.3                 
 [9] R6_2.5.1                    seqLogo_1.64.0 

# Variables

FAS coordinates were taken from here: https://www.ensembl.org/Homo_sapiens/Gene/Summary?db=core;g=ENSG00000026103;r=10:88953813-89029605

In [3]:
FAS_start <- 88953813

In [4]:
FAS_end <- 89029605

# Load data

The Chr10 fasta sequence was downloaded from here: https://www.ensembl.org/Homo_sapiens/Info/Index

In [5]:
chr10_sequence <- read.fasta("Homo_sapiens.GRCh38.dna_sm.chromosome.10.fa")
print(str(chr10_sequence))

List of 1
 $ 10: 'SeqFastadna' chr [1:133797422] "n" "n" "n" "n" ...
  ..- attr(*, "name")= chr "10"
  ..- attr(*, "Annot")= chr ">10 dna_sm:chromosome chromosome:GRCh38:10:1:133797422:1 REF"
NULL


In [6]:
# load DO peaks
# read in DO peaks
all_peaks <- read.csv("TableS1_diffPeaks_onPromoters.txt"
                     ,sep = "\t"
                     ,header = TRUE)
print(str(all_peaks))

'data.frame':	108335 obs. of  46 variables:
 $ cell_line           : chr  "OCI-Ly1" "OCI-Ly1" "OCI-Ly1" "OCI-Ly1" ...
 $ peakID              : chr  "1_181400_181555" "1_629896_629990" "1_804858_805040" "1_827273_827761" ...
 $ Chr                 : chr  "1" "1" "1" "1" ...
 $ Start               : int  181400 629896 804858 827273 865730 869766 915579 919766 920550 921144 ...
 $ End                 : int  181555 629990 805040 827761 865924 870035 915747 919853 920834 921348 ...
 $ mean_count          : num  21.2 1844.8 23 90.5 27.8 ...
 $ threshold_mean_count: int  20 20 20 20 20 20 20 20 20 20 ...
 $ filter_mean_count   : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...
 $ threshold_count     : int  20 20 20 20 20 20 20 20 20 20 ...
 $ filter_count        : logi  TRUE TRUE TRUE TRUE TRUE TRUE ...
 $ annoPackage         : chr  "ChIPseeker.v1.8.6" "ChIPseeker.v1.8.6" "ChIPseeker.v1.8.6" "ChIPseeker.v1.8.6" ...
 $ TxDb                : chr  "TxDb.Hsapiens.UCSC.hg38.knownGene" "TxDb.Hsapiens.UCSC.h

# FAS sequence

In [7]:
chr10_sequence <- paste(chr10_sequence$'10', collapse = '')
print(str(chr10_sequence))

 chr "nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn"| __truncated__
NULL


In [8]:
FAS_sequence <- toupper(substr(chr10_sequence
                      ,FAS_start
                      ,FAS_end
                      ))
print(str(FAS_sequence))

 chr "CTTGCTCTCTCTCGCCTGCTGCCATGTAAGACACGCCTCTTCCCCTTTTGCCATGAGTTTAAGTTTCTTGAGGCCTCCCCAGCCATGCAGAACTGTGAGTAAATTAAACCT"| __truncated__
NULL


In [9]:
rm(chr10_sequence)

# Run analysis

The JASPAR matrix for ETS1 can be found here: https://jaspar.genereg.net/search?q=ets1&collection=all&tax_group=all&tax_id=all&type=all&class=all&family=all&version=all

We will take the matrix MA0098.3

In [10]:
db <- "myMatrixDb.sqlite"
initializeJASPARDB(db, version="2020")

Warning message in result_fetch(res@ptr, n = n):
“SQL statements must be issued with dbExecute() or dbSendStatement() instead of dbGetQuery() or dbSendQuery().”
Warning message in result_fetch(res@ptr, n = n):
“SQL statements must be issued with dbExecute() or dbSendStatement() instead of dbGetQuery() or dbSendQuery().”
Warning message in result_fetch(res@ptr, n = n):
“SQL statements must be issued with dbExecute() or dbSendStatement() instead of dbGetQuery() or dbSendQuery().”
Warning message in result_fetch(res@ptr, n = n):
“SQL statements must be issued with dbExecute() or dbSendStatement() instead of dbGetQuery() or dbSendQuery().”
Warning message in result_fetch(res@ptr, n = n):
“SQL statements must be issued with dbExecute() or dbSendStatement() instead of dbGetQuery() or dbSendQuery().”
Warning message in result_fetch(res@ptr, n = n):
“SQL statements must be issued with dbExecute() or dbSendStatement() instead of dbGetQuery() or dbSendQuery().”
Warning message in result_fetch(re

[1] "Success"

In [11]:
suppressMessages(library(JASPAR2020))
opts <- list()
opts[["species"]] <- 9606
opts[["name"]] <- "ETS1"
opts[["all_versions"]] <- TRUE
PFMatrixList <- getMatrixSet(JASPAR2020, opts)
PFMatrixList
str(PFMatrixList)

PFMatrixList of length 2
names(2): MA0098.1 MA0098.3

Formal class 'PFMatrixList' [package "TFBSTools"] with 4 slots
  ..@ listData       :List of 2
  .. ..$ MA0098.1:Formal class 'PFMatrix' [package "TFBSTools"] with 7 slots
  .. .. .. ..@ ID           : chr "MA0098.1"
  .. .. .. ..@ name         : chr "ETS1"
  .. .. .. ..@ matrixClass  : chr "Tryptophan cluster factors"
  .. .. .. ..@ strand       : chr "+"
  .. .. .. ..@ bg           : Named num [1:4] 0.25 0.25 0.25 0.25
  .. .. .. .. ..- attr(*, "names")= chr [1:4] "A" "C" "G" "T"
  .. .. .. ..@ tags         :List of 8
  .. .. .. .. ..$ family       : chr "Ets-related factors"
  .. .. .. .. ..$ medline      : chr "1542566"
  .. .. .. .. ..$ tax_group    : chr "vertebrates"
  .. .. .. .. ..$ tfbs_shape_id: chr "1"
  .. .. .. .. ..$ type         : chr "SELEX"
  .. .. .. .. ..$ collection   : chr "CORE"
  .. .. .. .. ..$ species      : Named chr "Homo sapiens"
  .. .. .. .. .. ..- attr(*, "names")= chr "9606"
  .. .. .. .. ..$ acc          : chr "P14921"
  .. .. .. ..@ profileMatrix: int

In [12]:
pfmMA0098.3 <- PFMatrixList$MA0098.3

In [13]:
pwm <- toPWM(pfmMA0098.3, pseudocounts=0.8)
pwm

An object of class PWMatrix
ID: MA0098.3
Name: ETS1
Matrix Class: Tryptophan cluster factors
strand: +
Pseudocounts: 0.8
Tags: 
$comment
[1] "Data is from Taipale HTSELEX DBD (2013)"

$family
[1] "Ets-related factors"

$medline
[1] "20517297"

$remap_tf_name
[1] "ETS1"

$source
[1] "23332764"

$tax_group
[1] "vertebrates"

$type
[1] "HT-SELEX"

$unibind
[1] "1"

$collection
[1] "CORE"

$species
          9606 
"Homo sapiens" 

$acc
[1] "P14921"

Background: 
   A    C    G    T 
0.25 0.25 0.25 0.25 
Matrix: 
        [,1]      [,2]        [,3]       [,4]       [,5]       [,6]
A  1.5075544 -2.142626  -0.8733891 -11.723234 -11.718426   1.999677
C -2.1665656  1.753658   1.7843520  -4.995314 -11.718426 -11.711990
G -0.5598056 -1.420790  -6.7573900   1.988433   1.993241 -11.711990
T -1.9680984 -5.163593 -11.9273150 -11.723234  -5.787689 -11.711990
          [,7]       [,8]        [,9]      [,10]
A  1.562872652  0.6990088 -3.02077444  0.2595540
C -6.625232365 -5.0011692 -0.02360762 -0.7418837

In [14]:
subject <- DNAString(FAS_sequence)
print(subject)

75793-letter DNAString object
seq: CTTGCTCTCTCTCGCCTGCTGCCATGTAAGACACGC...GAATTGAAAGGGATGGCGGTCCCTCTTCCCAACATT


In [15]:
min_score <- 85

In [16]:
siteset <- searchSeq(pwm
                     , subject
                     , seqname="FAS_full_sequence"
                     , min.score= paste0(min_score,"%")
                     , strand="*")


In [17]:
# put in one table
FAS_GFF3 <- writeGFF3(siteset)
FAS_GFF3$TFMPvalue <- pvalues(siteset, type="TFMPvalue")
FAS_GFF3$start_true_coordinate <- FAS_GFF3$start + FAS_start
FAS_GFF3$end_true_coordinate <- FAS_GFF3$end + FAS_start
# sort by increasing pvalues
FAS_GFF3 <- FAS_GFF3[order(FAS_GFF3$TFMPvalue,decreasing = FALSE),]
print(head(FAS_GFF3))
print(str(FAS_GFF3))

              seqname source feature start   end    score strand frame
73  FAS_full_sequence   TFBS    TFBS 36230 36239 12.85471      +     .
215 FAS_full_sequence   TFBS    TFBS 67847 67856 12.76646      -     .
43  FAS_full_sequence   TFBS    TFBS 29539 29548 12.52331      +     .
202 FAS_full_sequence   TFBS    TFBS 58655 58664 12.31528      -     .
197 FAS_full_sequence   TFBS    TFBS 55343 55352 11.83518      -     .
75  FAS_full_sequence   TFBS    TFBS 37429 37438 11.33657      +     .
                                                      attributes    TFMPvalue
73  TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAGTA 2.479553e-05
215 TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAATG 2.670288e-05
43  TF=ETS1;class=Tryptophan cluster factors;sequence=ACCGGAAACC 2.765656e-05
202 TF=ETS1;class=Tryptophan cluster factors;sequence=GCCGGATGTG 3.910065e-05
197 TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAGTT 5.340576e-05
75  TF=ETS1;class=Tryptophan cluste

In [18]:
FAS_GFF3$padj <- p.adjust(FAS_GFF3$TFMPvalue,method = "fdr")
print(str(FAS_GFF3))

'data.frame':	222 obs. of  13 variables:
 $ seqname              : chr  "FAS_full_sequence" "FAS_full_sequence" "FAS_full_sequence" "FAS_full_sequence" ...
 $ source               : chr  "TFBS" "TFBS" "TFBS" "TFBS" ...
 $ feature              : chr  "TFBS" "TFBS" "TFBS" "TFBS" ...
 $ start                : int  36230 67847 29539 58655 55343 37429 51434 53194 52935 44596 ...
 $ end                  : int  36239 67856 29548 58664 55352 37438 51443 53203 52944 44605 ...
 $ score                : num  12.9 12.8 12.5 12.3 11.8 ...
 $ strand               : chr  "+" "-" "+" "-" ...
 $ frame                : chr  "." "." "." "." ...
 $ attributes           : chr  "TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAGTA" "TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAATG" "TF=ETS1;class=Tryptophan cluster factors;sequence=ACCGGAAACC" "TF=ETS1;class=Tryptophan cluster factors;sequence=GCCGGATGTG" ...
 $ TFMPvalue            : num  2.48e-05 2.67e-05 2.77e-05 3.91e-05 5.34e-05 ...

# Convert to .bed

BED format description can be found here: https://genome.ucsc.edu/FAQ/FAQformat.html#format1

In [19]:
bed <- FAS_GFF3[FAS_GFF3$padj < 0.05,]
str(bed)

'data.frame':	222 obs. of  13 variables:
 $ seqname              : chr  "FAS_full_sequence" "FAS_full_sequence" "FAS_full_sequence" "FAS_full_sequence" ...
 $ source               : chr  "TFBS" "TFBS" "TFBS" "TFBS" ...
 $ feature              : chr  "TFBS" "TFBS" "TFBS" "TFBS" ...
 $ start                : int  36230 67847 29539 58655 55343 37429 51434 53194 52935 44596 ...
 $ end                  : int  36239 67856 29548 58664 55352 37438 51443 53203 52944 44605 ...
 $ score                : num  12.9 12.8 12.5 12.3 11.8 ...
 $ strand               : chr  "+" "-" "+" "-" ...
 $ frame                : chr  "." "." "." "." ...
 $ attributes           : chr  "TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAGTA" "TF=ETS1;class=Tryptophan cluster factors;sequence=ACAGGAAATG" "TF=ETS1;class=Tryptophan cluster factors;sequence=ACCGGAAACC" "TF=ETS1;class=Tryptophan cluster factors;sequence=GCCGGATGTG" ...
 $ TFMPvalue            : num  2.48e-05 2.67e-05 2.77e-05 3.91e-05 5.34e-05 ...

In [20]:
bed$chrom = 10

In [21]:
bed$chromStart <- bed$start_true_coordinate

In [22]:
bed$chromEnd <- bed$end_true_coordinate

In [23]:
bed <- bed[,c("chrom"
             ,"chromStart"
             ,"chromEnd"
             )]

In [24]:
print(str(bed))

'data.frame':	222 obs. of  3 variables:
 $ chrom     : num  10 10 10 10 10 10 10 10 10 10 ...
 $ chromStart: num  8.9e+07 8.9e+07 8.9e+07 8.9e+07 8.9e+07 ...
 $ chromEnd  : num  8.9e+07 8.9e+07 8.9e+07 8.9e+07 8.9e+07 ...
NULL


# Export

In [25]:
write.table(bed
           ,"ETS1_binding_sites.bed"
           ,sep = "\t"
           ,row.names = FALSE
           ,col.names = FALSE
           ,quote = FALSE)

For UCSC browser add manually the following three lines:

browser position chr10: 88975070-89017059

track name=ETS1_binding_sites description=" " visibility=2

#chrom chromStart chromEnd